In [24]:
import torch
import torch.nn as nn

# MNIST 분류를 위한 MyRNN 설계 
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size): 
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        
        # 순차 데이터 처리를 위한 RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        
        # 분류를 위한 fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x): # x: [batch_size, seq_length, input_size] 
        # 초기 hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        # RNN 출력
        out, _ = self.rnn(x, h0)
        # 마지막 시퀀스의 출력만 사용 
        out = out[:, -1, :]
        # 마지막출력을 fully connected layer를 통해 최종 예측결과출력 
        out = self.fc(out)
        return out

In [30]:
import os
print(os.getcwd())

FileNotFoundError: [Errno 2] No such file or directory

In [28]:
import os
from torchvision import datasets, transforms

# 데이터 저장 경로 지정
data_dir = './MNIST_RNN_data'
print(os.getcwd())
# 디렉토리가 없다면 생성
try:
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
except OSError as e:
    print(f"Error: {data_dir}를 생성할 수 없습니다.")
    
train_dataset = datasets.MNIST(root=data_dir, train=True, download=True, transform=transforms.ToTensor()) 
test_dataset = datasets.MNIST(root=data_dir, train=False, download=True, transform=transforms.ToTensor())


FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
from torch.utils.data import DataLoader  # PyTorch에서 제공하는 DataLoader, 배치 크기 만큼의 데이터를 쉽게 불러올 수 있음
input_size = 28  # (28)*28
hidden_size = 128
output_size = 10  # 10개
learning_rate = 0.001
batch_size = 64
num_epochs = 5
# Create DataLoader for training and test datasets
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from torch import nn, optim

model = MyRNN(input_size, hidden_size, output_size)

# 손실 함수(loss function)을 교차 엔트로피(cross entropy)함수로 정의
criterion = nn.CrossEntropyLoss()

# 학습 방법을 Adam으로 정의
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    model.train() # 학습모드
    # loss 출력을 위한 변수 
    running_loss = 0.0
    # DataLoader가 배치 크기만큼의 데이터와 레이블을 각 반복마다 로드함 
    for images, labels in train_loader:
        images = images.view(-1, 28, 28)  # shape 조정 [batch_size, seq_length, input_size]
        # 미분은 누적되기 때문에 초기화 수행 
        optimizer.zero_grad()
        outputs = model(images)
        # outputs, labels를 통해 손실(loss) 계산 
        loss = criterion(outputs, labels)
        # 손실에 대한 미분(gradient) 역전파(back propagation) 
        loss.backward()
        # 가중치 업데이트 
        optimizer.step()

        # 손실 합산
        running_loss += loss.item()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}')



Epoch 1, Loss: 1.6372128359353872
Epoch 2, Loss: 0.6127837694434723
Epoch 3, Loss: 0.4456999868805856
Epoch 4, Loss: 0.3777586601928734
Epoch 5, Loss: 0.3174789589085082


In [ ]:
# Evaluation on test set
correct_prediction = 0
total_prediction = 0

model.eval() 
# 테스트모드(드롭아웃(dropout)이나 배치 정규화(batch normalization)와 같은 학습 중에만 활성화되는 기능을 포함하고 있다면, 추론(inference) 시에는 이러한 기능을 비활성화)

# 테스트에서는 미분 계산이 필요 없음 #파이토치 자동미분 차단
with torch.no_grad():
    # 테스트 데이터셋에 대한 데이터로더 
    for images, labels in test_loader:
        images = images.view(-1, 28, 28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_prediction += labels.size(0)
        # 예측(pred)과 정답(labels)가 일치하는 수만큼 합산 
        correct_prediction += (predicted == labels).sum().item()
print('Accuracy of RNN in the 10000 test images: {:.2f}%'.format(100 * correct_prediction / total_prediction))

Accuracy of RNN in the 10000 test images: 96.15%
